In [ ]:
"""
Классификация типа покрытия (asphalt vs cobblestones) по аудио-сегментам 0.1с
- Устойчивое чтение WAV: soundfile -> librosa -> scipy (fallback)
- Фиксация длины 0.1с от фактического SR (без принудительного ресэмплинга)
- Нормализация амплитуды, ослабленный energy gate, регистронезависимые .wav
- Параллельная обработка joblib
- Полные отчёты (precision/recall/f1/support) для train и test (Linear и RBF SVM)
"""

import os
import sys
import numpy as np
import librosa
import soundfile as sf
from scipy.io import wavfile as scipy_wav
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score, accuracy_score
from joblib import Parallel, delayed
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

# НАСТРОЙКИ 
# ВАЖНО: укажите путь к корневой папке датасета
# Примеры:
# - Локально: "raw_audio"
# - Kaggle: "/kaggle/input/<slug>/raw_audio"
ROOT_DATA_DIR = "raw_audio"

DATASETS = [
    os.path.join(ROOT_DATA_DIR, "asphalt_dry"),
    os.path.join(ROOT_DATA_DIR, "asphalt_wet"),
    os.path.join(ROOT_DATA_DIR, "cobblestones_dry"),
    os.path.join(ROOT_DATA_DIR, "cobblestones_wet"),
]
POSSIBLE_PAV_TYPES = {"asphalt", "cobblestones"}

# АУДИО: длительность сегмента ровно 0.1с; используем фактический SR файла
TARGET_SR = None # не форсируем ресэмплинг
SEGMENT_SECONDS = 0.1
ENERGY_THR = 1e-8 # ослабленный порог для коротких сегментов

# Параметры признаков (под короткое окно)
N_MFCC = 13
N_FFT = 512
HOP_LENGTH = 256
N_MELS = 40

# Параметры выполнения
N_JOBS = -1 # -1 => все ядра CPU
MAX_FILES_PER_FOLDER = None # например 2000 для быстрого прогона
PRINT_FIRST_ERRORS = 10 # 0 чтобы не печатать примеры ошибок
USE_TQDM = True # прогресс-бар включён

#  ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ 

def parse_label(folder_name: str):
    fl = folder_name.lower()
    for p in POSSIBLE_PAV_TYPES:
        if p in fl:
            return p
    return None

def safe_load(file_path: str, target_sr: int | None = TARGET_SR):
    """
    Надёжная загрузка WAV: soundfile -> librosa -> scipy.
    Возвращает (y, sr, err_str|None).
    """
    # 1) soundfile
    try:
        y, sr = sf.read(file_path, always_2d=False)
        if y is None or (hasattr(y, '__len__') and len(y) == 0):
            raise ValueError('empty (soundfile)')
        if hasattr(y, 'ndim') and y.ndim > 1:
            y = y.mean(axis=1)
        y = y.astype(np.float32)
        if target_sr and sr != target_sr:
            y = librosa.resample(y, orig_sr=sr, target_sr=target_sr, res_type='kaiser_fast')
            sr = target_sr
        return y, sr, None
    except Exception as e1:
        last_err = f"sf:{type(e1).__name__}:{e1}"
    # 2) librosa
    try:
        y, sr = librosa.load(file_path, sr=target_sr, mono=True, res_type='kaiser_fast')
        if y is None or len(y) == 0:
            raise ValueError('empty (librosa)')
        return y.astype(np.float32), sr, None
    except Exception as e2:
        last_err = last_err + f"|lb:{type(e2).__name__}:{e2}"
    # 3) scipy
    try:
        sr0, y = scipy_wav.read(file_path)
        if y is None or len(y) == 0:
            raise ValueError('empty (scipy)')
        y = y.astype(np.float32)
        if y.ndim > 1:
            y = y.mean(axis=1)
        if target_sr and sr0 != target_sr:
            y = librosa.resample(y, orig_sr=sr0, target_sr=target_sr, res_type='kaiser_fast')
            sr0 = target_sr
        return y.astype(np.float32), sr0, None
    except Exception as e3:
        last_err = last_err + f"|sp:{type(e3).__name__}:{e3}"
    return None, None, last_err

def expected_samples(sr: int) -> int:
    return int(round(sr * SEGMENT_SECONDS))

def fix_length_0p1sec(y: np.ndarray, sr: int) -> np.ndarray:
    """
    Приводит длину к ровно 0.1с, подрезая/дополняя нулями по фактическому SR.
    """
    exp = expected_samples(sr)
    n = len(y)
    if n > exp:
        y = y[:exp]
    elif n < exp:
        y = np.pad(y, (0, exp - n), mode='constant')
    return y

def energy_gate(y: np.ndarray, thr: float = ENERGY_THR) -> bool:
    """
    Отсев почти «немых» сегментов по средней энергии.
    """
    return (np.mean(y**2) >= thr)

def extract_features(y: np.ndarray, sr: int) -> np.ndarray:
    """
    MFCC + Delta (mean/std) + spectral centroid, ZCR, RMS (mean/std) для коротких окон.
    """
    # Нормализация амплитуды
    m = np.max(np.abs(y))
    if m > 0:
        y = y / (m + 1e-9)

    feats = []
    mfcc = librosa.feature.mfcc(
        y=y, sr=sr, n_mfcc=N_MFCC,
        n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS
    )
    mfcc_d = librosa.feature.delta(mfcc, order=1)

    for feat in [mfcc, mfcc_d]:
        feats.extend(np.mean(feat, axis=1).tolist())
        feats.extend(np.std(feat, axis=1).tolist())

    sc = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, center=True)[0]
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=N_FFT, hop_length=HOP_LENGTH, center=True)[0]
    rms = librosa.feature.rms(y=y, frame_length=N_FFT, hop_length=HOP_LENGTH, center=True)[0]

    for arr in [sc, zcr, rms]:
        feats.append(float(np.mean(arr)))
        feats.append(float(np.std(arr)))

    v = np.array(feats, dtype=np.float32)
    v = np.nan_to_num(v, nan=0.0, posinf=0.0, neginf=0.0)  # защитимся от NaN/Inf
    return v

def process_file(file_path: str, label: str):
    y, sr, err = safe_load(file_path)
    if y is None:
        return None, f"{os.path.basename(file_path)} -> load_failed: {err}"
    y = fix_length_0p1sec(y, sr)
    if not energy_gate(y):
        return None, f"{os.path.basename(file_path)} -> too_silent"
    try:
        v = extract_features(y, sr)
        if not np.isfinite(v).all():
            return None, f"{os.path.basename(file_path)} -> nan_or_inf_in_features"
        return (v, label), None
    except Exception as e:
        return None, f"{os.path.basename(file_path)} -> feature_error: {type(e).__name__}: {e}"

def load_dataset():
    X, y = [], []
    for d in DATASETS:
        folder = os.path.basename(d)
        print(f"Обработка: {d}")
        if not os.path.exists(d):
            print("Папка не найдена, пропущена")
            continue
        label = parse_label(folder)
        if label is None:
            print("Не удалось определить метку, пропущена")
            continue
        files = [f for f in os.listdir(d) if f.lower().endswith('.wav')]
        if len(files) == 0:
            print("Нет .wav/.WAV файлов")
            continue
        if MAX_FILES_PER_FOLDER is not None:
            files = files[:MAX_FILES_PER_FOLDER]
        print(f"Файлов для обработки: {len(files)}")
        print(f"Метка: {label}")
        paths = [os.path.join(d, f) for f in files]

        iterator = paths
        if USE_TQDM:
            iterator = tqdm(paths, desc=f"  {folder}", ncols=80, leave=False, mininterval=0.2)

        results = Parallel(n_jobs=N_JOBS)(
            delayed(process_file)(p, label) for p in iterator
        )

        ok = 0
        errs_local = []
        for res, err in results:
            if err is None:
                v, lbl = res
                X.append(v)
                y.append(lbl)
                ok += 1
            else:
                if len(errs_local) < PRINT_FIRST_ERRORS:
                    errs_local.append(err)

        print(f"Успешно обработано: {ok}/{len(paths)}")
        if errs_local:
            print("Примеры причин отказа (первые):")
            for e in errs_local:
                print("   -", e)
    return np.array(X, dtype=np.float32), np.array(y)

# ===================== ОБУЧЕНИЕ И ОЦЕНКА =====================

def train_and_evaluate(X: np.ndarray, y: np.ndarray):
    if len(X) == 0:
        print("Не удалось извлечь признаки ни из одного файла")
        sys.exit(1)

    print(f"Всего сегментов: {len(X)} | Размер признака: {X.shape[1]}")
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    X_tr, X_te, y_tr, y_te = train_test_split(X, y_enc, test_size=0.2, stratify=y_enc, random_state=42)

    # 1) Linear SVM — отчёты для TRAIN и TEST
    print("================= Linear SVM =================")
    lin = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', LinearSVC(C=1.0, class_weight='balanced', max_iter=2000, random_state=42))
    ])
    lin.fit(X_tr, y_tr)
    y_tr_lin = lin.predict(X_tr)
    y_te_lin = lin.predict(X_te)
    print("Отчёт (Linear SVM — TRAIN):")
    print(classification_report(y_tr, y_tr_lin, target_names=le.classes_))
    print("Отчёт (Linear SVM — TEST):")
    print(classification_report(y_te, y_te_lin, target_names=le.classes_))
    print(f"Метрики: acc_train={accuracy_score(y_tr, y_tr_lin):.4f} f1w_train={f1_score(y_tr, y_tr_lin, average='weighted'):.4f} | acc_test={accuracy_score(y_te, y_te_lin):.4f} f1w_test={f1_score(y_te, y_te_lin, average='weighted'):.4f}")

    # 2) RBF SVM — отчёты для TRAIN и TEST
    print("================= RBF SVM =================")
    rbf = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf', C=3.0, gamma='scale', class_weight='balanced', cache_size=500, random_state=42))
    ])
    rbf.fit(X_tr, y_tr)
    y_tr_rbf = rbf.predict(X_tr)
    y_te_rbf = rbf.predict(X_te)
    print("Отчёт (RBF SVM — TRAIN):")
    print(classification_report(y_tr, y_tr_rbf, target_names=le.classes_))
    print("Отчёт (RBF SVM — TEST):")
    print(classification_report(y_te, y_te_rbf, target_names=le.classes_))
    print(f"Метрики: acc_train={accuracy_score(y_tr, y_tr_rbf):.4f} f1w_train={f1_score(y_tr, y_tr_rbf, average='weighted'):.4f} | acc_test={accuracy_score(y_te, y_te_rbf):.4f} f1w_test={f1_score(y_te, y_te_rbf, average='weighted'):.4f}")

def main():
    if not os.path.exists(ROOT_DATA_DIR):
        print("Укажите корректный ROOT_DATA_DIR")
        print("Текущее значение:", ROOT_DATA_DIR)
        sys.exit(1)
    print("Старт обработки аудио-сегментов 0.1с…")
    X, y = load_dataset()
    print("Переходим к обучению и оценке моделей…")
    train_and_evaluate(X, y)
    print("Готово.")

if __name__ == '__main__':
    main()
